# Imports

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
df_train_raw = pd.read_parquet('data/train.parquet')

# Step 0: EDA (skrub)

In [ ]:
from skrub import TableReport

TableReport(df_train_raw)

# Step 1: Aggregation per user and per day

In [ ]:
warnings.filterwarnings('ignore')

def step_1_5_session_features(df_raw):
    """
    Calculates session-based features from raw logs.
    A session is defined by a gap of > 30 minutes between events.
    """
    df = df_raw.sort_values(['userId', 'ts']).copy()

    # Calculate time difference between consecutive events
    df['prev_ts'] = df.groupby('userId')['ts'].shift(1)
    df['time_diff'] = (df['ts'] - df['prev_ts']) / 1000 / 60 # in minutes

    # New session starts if gap > 30 mins or new user
    df['new_session'] = ((df['time_diff'] > 30) | (df['time_diff'].isnull())).astype(int)
    df['sessionId_generated'] = df.groupby('userId')['new_session'].cumsum()

    # Aggregate per session
    session_stats = df.groupby(['userId', 'sessionId_generated']).agg({
        'ts': ['min', 'max', 'count'], # Start, End, # Items
        'page': lambda x: (x == 'Error').sum() # Errors per session
    })
    session_stats.columns = ['session_start', 'session_end', 'items_per_session', 'errors_per_session']

    # Calculate Session Duration
    session_stats['session_duration_min'] = (session_stats['session_end'] - session_stats['session_start']) / 1000 / 60

    # Aggregate per User per Day
    session_stats['date'] = pd.to_datetime(session_stats['session_start'], unit='ms').dt.floor('D')

    daily_session_features = session_stats.groupby(['userId', 'date']).agg({
        'session_duration_min': 'mean',
        'items_per_session': 'mean',
        'errors_per_session': 'mean'
    }).rename(columns={
        'session_duration_min': 'avg_session_duration_daily',
        'items_per_session': 'avg_items_per_session_daily',
        'errors_per_session': 'avg_errors_per_session_daily'
    }).reset_index()

    return daily_session_features

In [ ]:
warnings.filterwarnings('ignore')

def step_1_daily_aggregation(df, df_session_features=None):
    """
    Transforms raw logs into a Daily User Activity df (grouped by userid and day)

    Logic:
    1. Converts timestamps to daily dates
    2. Filter out the target event
    3. Feature engineers some specific page events (NextSong, Errors, etc.) into features
    4. Aggregates daily metrics like listening time or session count
    5. Encodes the 'level' column (paid/free) to binary (paid=1, free=0)
    """

    # 1.
    df['date'] = pd.to_datetime(df['ts'], unit='ms').dt.floor('D')

    # 2.
    df_features = df[df['page'] != 'Cancellation Confirmation']

    # 3.
    relevant_pages = [
        'NextSong', 'Thumbs Up', 'Thumbs Down',
        'Error', 'Add Friend', 'Roll Advert', 'Upgrade', 'Downgrade',
        'Add to Playlist', 'Settings', 'Help'
    ]
    df_relevant = df_features[df_features['page'].isin(relevant_pages)]

    daily_page_counts = df_relevant.pivot_table(
        index=['userId', 'date'],
        columns='page',
        values='ts',
        aggfunc='count',
        fill_value=0
    )
    daily_page_counts.columns = [f'daily_{col.replace(" ", "")}' for col in daily_page_counts.columns]

    # 4.
    daily_stats = df_features.groupby(['userId', 'date']).agg({
        'length': 'sum',              # listening time
        'sessionId': 'nunique',       # session count
        'level': 'last',              # paid/Free
        'registration': 'max',        # registration time
        'artist': 'nunique',          # diversity
    }).rename(columns={
        'length': 'daily_listen_time',
        'sessionId': 'daily_sessions',
        'level': 'status_level',
        'artist': 'daily_unique_artists'})

    # merging both df
    df_daily = pd.concat([daily_page_counts, daily_stats], axis=1).reset_index()

    if df_session_features is not None:
        df_daily = df_daily.merge(df_session_features, on=['userId', 'date'], how='left').fillna(0)

    # 5.
    df_daily['is_paid'] = df_daily['status_level'].apply(lambda x: 1 if x == 'paid' else 0)
    df_daily.drop('status_level', axis=1, inplace=True)

    return df_daily

In [ ]:
df_sessions = step_1_5_session_features(df_train_raw)
df_daily_train = step_1_daily_aggregation(df_train_raw, df_session_features=df_sessions)

In [ ]:
df_daily_train.head()

# Step 2: Feature Engineering

In [ ]:
def step_2_feature_engineering(df_daily):
    """
    Feature Engineering focusing on rolling trends and velocity.
    """
    df_daily = df_daily.sort_values(['userId', 'date'])

    # 1. Standard Rolling Windows (Shifted to avoid leakage)
    features_to_roll = [
        'daily_NextSong', 'daily_Error', 'daily_ThumbsDown', 'daily_ThumbsUp',
        'daily_RollAdvert', 'daily_listen_time', 'daily_AddFriend', 'daily_Help',
        'daily_AddtoPlaylist', 'daily_Downgrade', 'daily_Upgrade', 'daily_sessions',
        'daily_unique_artists'
    ]

    for col in features_to_roll:
        grouped = df_daily.groupby('userId')[col]

        # Windows: 3d (immediate), 7d (short), 14d (medium), 30d (long)
        df_daily[f'{col}_last_3d'] = grouped.transform(lambda x: x.shift(1).rolling(3, min_periods=1).sum())
        df_daily[f'{col}_last_7d'] = grouped.transform(lambda x: x.shift(1).rolling(7, min_periods=1).sum())
        df_daily[f'{col}_last_14d'] = grouped.transform(lambda x: x.shift(1).rolling(14, min_periods=1).sum())
        df_daily[f'{col}_last_30d'] = grouped.transform(lambda x: x.shift(1).rolling(30, min_periods=1).sum())

        # 1. Immediate Trend: Is activity dropping in the last 3 days compared to the last 2 weeks?
        # A ratio < 1.0 means they are slowing down --> high risk
        df_daily[f'{col}_trend_3d_vs_14d'] = df_daily[f'{col}_last_3d'] / (df_daily[f'{col}_last_14d'] / 4.6 + 1)

        # 2. Weekly Trend: Is this week worse than last month average?
        df_daily[f'{col}_trend_7d_vs_30d'] = df_daily[f'{col}_last_7d'] / (df_daily[f'{col}_last_30d'] / 4.2 + 1)

        # 3. Sudden Drop (Delta): Explicitly calculate the drop
        df_daily[f'{col}_drop_7d'] = (df_daily[f'{col}_last_7d'] / 7) - (df_daily[f'{col}_last_30d'] / 30)

        # Calculate intermediate rolling window for math
        grouped = df_daily.groupby('userId')[col]
        roll_21d = grouped.transform(lambda x: x.shift(1).rolling(21, min_periods=1).sum())

        # Activity from day -30 to day -21 (approx)
        slice_1_val = df_daily[f'{col}_last_30d'] - roll_21d

        # ratio: current Week vs 4 weeks Ago
        df_daily[f'{col}_current_vs_month_ago'] = df_daily[f'{col}_last_7d'] / (slice_1_val + 1)

        # rolling variance: only for important features
        if col in ['daily_NextSong', 'daily_AddtoPlaylist']:
            df_daily[f'{col}_last_7d_var'] = grouped.transform(lambda x: x.shift(1).rolling(7, min_periods=1).var().fillna(0))

    # 2. recency & gaps
    df_daily['prev_date'] = df_daily.groupby('userId')['date'].shift(1)
    df_daily['days_since_last_active'] = (df_daily['date'] - df_daily['prev_date']).dt.days.fillna(500) # 500 for never active
    df_daily.drop('prev_date', axis=1, inplace=True)

    # variance of inactivity periods (lifetime)
    df_daily['time_since_last_action_var'] = df_daily.groupby('userId')['days_since_last_active'].transform(
    lambda x: x.shift(1).expanding().var().fillna(0))

    # 3. registration lifetime
    reg_date = pd.to_datetime(df_daily['registration']).dt.floor('D')
    df_daily['days_since_reg'] = (df_daily['date'] - reg_date).dt.days

    # 4. Ratios (Intensity)
    # Errors per hour (Frustration index)
    df_daily['errors_per_hour'] = df_daily['daily_Error_last_7d'] / (df_daily['daily_listen_time_last_7d'] / 3600 + 1)

    # Songs per Session (Engagement depth)
    df_daily['songs_per_session'] = df_daily['daily_NextSong_last_7d'] / (df_daily['daily_sessions_last_7d'] + 1)

    # Clean up
    cols_to_drop = ['raw_agent', 'userAgent', 'summary', 'artist']
    for c in cols_to_drop:
        if c in df_daily.columns:
            df_daily.drop(c, axis=1, inplace=True)

    return df_daily.fillna(0)

In [ ]:
df_features_train = step_2_feature_engineering(df_daily_train)

# Step 3: Regroup dataset for a static target (1 row per user)

In [ ]:
def step_3_generate_stacked_dataset(df_features, df_raw_logs, cutoff_dates):
    """
    Generates a stacked dataset by creating multiple snapshots to have more data.
    Iterates through cutoff dates, creates a snapshot for each, and stacks them into one large training set.
    """
    stacked_snapshots = []

    print(f"generating stacked dataset across {len(cutoff_dates)} snapshots")

    for cutoff_date in cutoff_dates:
        cutoff_ts = pd.Timestamp(cutoff_date)
        prediction_window_end = cutoff_ts + pd.Timedelta(days=10)

        # 1. filter features: keep only activity BEFORE this specific cutoff
        df_history = df_features[df_features['date'] <= cutoff_ts].copy()

        # skip if no data available yet
        if df_history.empty:
            continue

        # 2. take the last known state for each user as of this specific cutoff
        df_snapshot = df_history.sort_values('date').groupby('userId').tail(1)

        # 3. define target: look ahead (10 days after cutoff, just like the final test)
        # find users who cancelled in the [cutoff, cutoff+10d] window
        churn_events = df_raw_logs[df_raw_logs['page'] == 'Cancellation Confirmation']
        churn_events['churn_date'] = pd.to_datetime(churn_events['ts'], unit='ms').dt.floor('D')

        churners_in_window = churn_events[
            (churn_events['churn_date'] > cutoff_ts) &
            (churn_events['churn_date'] <= prediction_window_end)
        ]['userId'].unique()

        # 4. assign labels (1 = churn in window, 0 = safe)
        df_snapshot['target_churn'] = df_snapshot['userId'].isin(churners_in_window).astype(int)

        # 5. add date (useful for debugging or splitting later)
        df_snapshot['snapshot_date'] = cutoff_ts

        stacked_snapshots.append(df_snapshot)
        print(f"--> Snapshot {cutoff_date}: {len(df_snapshot)} users | {df_snapshot['target_churn'].sum()} churners")

    df_stacked = pd.concat(stacked_snapshots, axis=0, ignore_index=True)

    print(f"\nstacked sataset created. Total rows: {len(df_stacked)}")
    print(f"Overall class balance: {df_stacked['target_churn'].value_counts().to_dict()}")

    return df_stacked

In [ ]:
# we generate a list of dates every 2 days from Oct 7th to Nov 1st
# this gives 13 snapshots (approx 3.5 per week)
cutoff_dates = pd.date_range(start='2018-10-7', end='2018-11-01', freq='2D').strftime('%Y-%m-%d').tolist()

print(f"Cutoff Dates Selected: {cutoff_dates}")

df_stacked_train = step_3_generate_stacked_dataset(df_features_train, df_train_raw, cutoff_dates)

In [ ]:
df_stacked_train.head()

# Step 4: Fine-tuning model with Optuna

In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score, GroupKFold
from sklearn.feature_selection import SelectFromModel

def optimize_with_optuna(df_final, n_trials=50, subsample_ratio=0.4):
    """
    Optimizes hyperparameters using a random sample of the data to save training time.

    Args:
        df_final: the full stacked dataframe
        n_trials: number of optuna trials
        subsample_ratio: fraction of data to use for optimization (0.4 = 40%).
    """
    # 1. prepare data & downsample for speed
    # we maintain the class ratio while downsampling

    if subsample_ratio < 1.0:
        print(f"--> downsampling dataset to {subsample_ratio*100}% for fast optimization")
        # Stratified sample to keep churn rate consistent
        df_optim_sample = df_final.groupby('target_churn', group_keys=False).apply(
            lambda x: x.sample(frac=subsample_ratio, random_state=5)
        )
    else:
        df_optim_sample = df_final.copy()

    print(f"optimization data shape: {df_optim_sample.shape}")

    groups = df_optim_sample['userId']

    # drop metadata for optuna optimization
    cols_to_drop = ['userId', 'date', 'registration', 'target_churn', 'snapshot_date']
    existing_drop = [c for c in cols_to_drop if c in df_optim_sample.columns]

    X = df_optim_sample.drop(columns=existing_drop)
    y = df_optim_sample['target_churn']

    # calculate scale pos weight based on the sample
    ratio = (y == 0).sum() / (y == 1).sum()
    print(f"sample class imbalance ratio: {ratio:.2f}")

    # 2. feature selection (from a simplified model)
    print("--> Running Feature Selection")
    selector = xgb.XGBClassifier(
        learning_rate=0.03,
        max_depth=4,
        random_state=5,
        n_jobs=-1
    )
    selector.fit(X, y)

    selection = SelectFromModel(selector, threshold='0.6*mean', prefit=True)
    X_selected = pd.DataFrame(selection.transform(X), columns=X.columns[selection.get_support()])

    final_features = X_selected.columns.tolist()
    print(f"selected {len(final_features)} features")

    # 3. Define Objective
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 300, 600),
            'max_depth': trial.suggest_int('max_depth', 3, 6),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
            'subsample': trial.suggest_float('subsample', 0.6, 0.9),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
            'min_child_weight': trial.suggest_float('min_child_weight', 1, 5),
            'scale_pos_weight': trial.suggest_float('scale_pos_weight', 5, ratio),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 5.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 5.0, 15.0),
            'gamma': trial.suggest_float('gamma', 0, 2.0),

            # Speed optimizations
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'tree_method': 'hist',
            'n_jobs': -1,
            'random_state': 5
        }

        gkf = GroupKFold(n_splits=5)

        model = xgb.XGBClassifier(**params)

        scores = cross_val_score(model, X_selected, y, cv=gkf, groups=groups, scoring='roc_auc', n_jobs=-1)
        return scores.mean()

    # 4. Run Optimization
    print(f"\n--> starting optuna ({n_trials} trials)")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='maximize')

    # enqueue known good params to speed up convergence
    study.enqueue_trial({
        'max_depth': 3,
        'learning_rate': 0.03,
        'subsample': 0.8,
        'colsample_bytree': 0.7
    })

    study.optimize(objective, n_trials=n_trials)

    print(f"--> Best AUC on Sample: {study.best_value:.4f}")

    best_params = study.best_params.copy()
    # add back the standard static params
    best_params.update({
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'tree_method': 'hist',
        'n_jobs': -1,
        'random_state': 5
    })

    return best_params, final_features

best_params_dict, final_features = optimize_with_optuna(df_stacked_train, n_trials=50, subsample_ratio=1)

In [ ]:
best_params_dict

# Step 5: 5-fold grouped prediction + generate output

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score

def train_stacked_ensemble(df_final, params, n_splits=5):
    """
    Trains a robust ensemble and returns OOF predictions for plotting.
    """
    cols_to_drop = ['userId', 'date', 'registration', 'target_churn', 'snapshot_date']
    existing_drop = [c for c in cols_to_drop if c in df_final.columns]

    X = df_final.drop(columns=existing_drop)
    y = df_final['target_churn']
    groups = df_final['userId']

    feature_names = X.columns.tolist()

    gkf = GroupKFold(n_splits=n_splits)
    models = []
    oof_preds = np.zeros(len(X))

    print(f"starting {n_splits}-fold group training on stacked data")

    for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups=groups)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train, verbose=False)

        val_probs = model.predict_proba(X_val)[:, 1]
        oof_preds[val_idx] = val_probs
        models.append(model)

        score = roc_auc_score(y_val, val_probs)
        print(f"fold {fold+1} AUC: {score:.4f}")

    best_thresh = 0.5
    best_f1 = 0
    for thresh in np.arange(0.2, 0.6, 0.01):
        preds = (oof_preds >= thresh).astype(int)
        score = f1_score(y, preds)
        if score > best_f1:
            best_f1 = score
            best_thresh = thresh

    print(f"\noverall OOF AUC: {roc_auc_score(y, oof_preds):.4f}")
    print(f"OOF 'Natural' Threshold: {best_thresh:.3f} (F1: {best_f1:.4f})")

    # we return 'y' and 'oof_preds' so we can plot curves later
    return models, feature_names, y, oof_preds

def generate_final_submission_with_target_rate(models, test_path, feature_cols, target_churn_rate=0.4):
    """
    Generates submission and dynamically finds the threshold
    to match the exact 'target_churn_rate' (e.g. 0.4)
    """
    df_test_raw = pd.read_parquet(test_path)

    # reuse pipeline
    df_sessions = step_1_5_session_features(df_test_raw)
    df_daily_test = step_1_daily_aggregation(df_test_raw, df_session_features=df_sessions)
    df_features_test = step_2_feature_engineering(df_daily_test)

    # snapshot last day
    df_test_last = df_features_test.sort_values('date').groupby('userId').tail(1)

    # align columns
    X_test = df_test_last.copy()
    for col in feature_cols:
        if col not in X_test.columns:
            X_test[col] = 0
    X_test = X_test[feature_cols]

    # predict
    print(f"predicting with {len(models)}-model ensemble")
    avg_probs = np.zeros(len(X_test))
    for model in models:
        avg_probs += model.predict_proba(X_test)[:, 1]
    avg_probs = avg_probs / len(models)

    # CALCULATE THRESHOLD FOR EXACT RATE
    # to get top 40%, we need the (100% - 40%) = 60th percentile
    quantile = 1 - target_churn_rate
    forced_threshold = np.quantile(avg_probs, quantile)

    print(f"\ntarget churn rate: {target_churn_rate:.2%}")
    print(f"--> calculated threshold: {forced_threshold:.5f}")

    # create submission
    submission = pd.DataFrame({
        'id': df_test_last['userId'],
        'target_prob': avg_probs,
        'target_binary': (avg_probs >= forced_threshold).astype(int)
    })

    sub_name_bin = f'submissions/submission_final_binary.csv'
    sub_name_prob = f'submissions/submission_final_prob.csv'

    submission[['id', 'target_binary']].rename(columns={'target_binary': 'target'}).to_csv(sub_name_bin, index=False)
    submission[['id', 'target_prob']].rename(columns={'target_prob': 'target'}).to_csv(sub_name_prob, index=False)

    print(f"saved: {sub_name_bin}")
    print(f"final predicted churn rate: {submission['target_binary'].mean():.2%}")

    return submission

import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

def plot_performance_curves(y_true, y_scores):
    """
    plots ROC and precision-recall curves side by side.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # 1. ROC curve
    RocCurveDisplay.from_predictions(y_true, y_scores, ax=ax1, name="our XGBoost model")
    ax1.plot([0, 1], [0, 1], "k--", label="random chance")
    ax1.set_title("ROC curve")
    ax1.legend()
    ax1.grid(alpha=0.3)

    # 2. precision-recall curve
    PrecisionRecallDisplay.from_predictions(y_true, y_scores, ax=ax2, name="our XGBoost model")
    ax2.set_title("precision-recall curve")
    ax2.legend()
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    plt.show();

In [ ]:
import pandas as pd
import numpy as np

def plot_cumulative_gain(y_true, y_scores):
    """
    plots the cmulative gain curve ("banana curve").
    shows what % of total churners we catch by targeting the top X% of users
    (in terms of potential churners)
    """
    data = pd.DataFrame({'true': y_true, 'score': y_scores})
    data = data.sort_values('score', ascending=False).reset_index(drop=True)

    # calculate cumulative stats
    data['cumulative_churners'] = data['true'].cumsum()
    total_churners = data['true'].sum()

    # calculate percentages (x and y axes)
    data['percent_users_contacted'] = (data.index + 1) / len(data) * 100
    data['percent_churners_found'] = data['cumulative_churners'] / total_churners * 100

    plt.figure(figsize=(8, 5))
    plt.plot(data['percent_users_contacted'], data['percent_churners_found'],
             label='our XGBoost model', color='tab:blue', linewidth=2)

    # baseline (random guessing)
    plt.plot([0, 100], [0, 100], 'k--', label='Random Guessing', alpha=0.5)

    # formatting
    plt.title('Cumulative Gains Curve (the "banana" chart)')
    plt.xlabel('% of Users Contacted')
    plt.ylabel('% of Total Churners Found')
    plt.grid(alpha=0.3)
    plt.legend()

    # highlight the top 40% point (the target churn rate)
    top_40_idx = int(len(data) * 0.40)
    recall_at_40 = data.loc[top_40_idx, 'percent_churners_found']

    plt.plot(40, recall_at_40, 'ro')
    plt.annotate(f"top 40% -> Captures {recall_at_40:.1f}% of churners",
                 xy=(40, recall_at_40), xytext=(45, recall_at_40-15),
                 arrowprops=dict(facecolor='black', shrink=0.05))

    plt.tight_layout()
    plt.show()

In [ ]:
# EXECUTION

# 1. get params from step 4
final_params = best_params_dict.copy()
final_params['random_state'] = 5
final_params['n_jobs'] = -1

# 2. train models & get OOF predictions
models, feature_names, y_true, oof_preds = train_stacked_ensemble(df_stacked_train, final_params)

# 3. plot curves
print("\ngenerating performance plots:")
plot_performance_curves(y_true, oof_preds)
plot_cumulative_gain(y_true, oof_preds)

# 4. generate Submission with exact 40% churn
sub = generate_final_submission_with_target_rate(
    models,
    test_path='data/test.parquet',
    feature_cols=feature_names,
    target_churn_rate=0.4
)